In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from collections import defaultdict

sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(10,5)})

In [ ]:
chains = pd.read_csv('/Users/mario/code/erp/chains_output/chains_mi.csv').dropna()
dials = pd.read_csv('/Users/mario/Downloads/data/pb_gpt2-ft.csv').dropna()

In [ ]:
chains.head()

In [ ]:
chains.keys()

In [ ]:
dials.keys()

In [ ]:
chains.iloc[0, :].values

In [ ]:
hs = []
hsc = []
not_found = 0
found_multiple = defaultdict(list)

new_df = []

for i in range(len(chains)):
    dial_id = chains.iloc[i, 0]
    speaker = chains.iloc[i, 1]
    round_number = chains.iloc[i, 3]
    tokens = chains.iloc[i, 6]

    utterance_in_dialogue = dials[
        (dials['dialogue_id'] == dial_id)
        & (dials['speaker'] == speaker)
        & (dials['round_number'] == round_number)
        & (dials['tokens'] == tokens)
    ]
    
    if len(utterance_in_dialogue) == 0:
        not_found += 1
        continue
        
#     assert len(utterance_in_dialogue) == 1, f'Found {len(utterance_in_dialogue)} utterances'
    
    if len(utterance_in_dialogue) > 1:
#         print(dial_id, round_number, speaker, tokens)
#         print()
#         print(utterance_in_dialogue)

        for j, row in utterance_in_dialogue.iterrows():
            
            position_in_round = row['position_in_round']
#             print(position_in_round)
            if position_in_round in found_multiple[(dial_id, speaker, round_number, tokens)]:
#                 print('Already processed')
                continue
            else:
                found_multiple[(dial_id, speaker, round_number, tokens)].append(position_in_round)
#                 print(f'This is position {position_in_round}')
                utterance_in_dialogue = pd.DataFrame(row)
                hs = row['normalised_h']
                hsc = row['normalised_h_doc']
    else:
                
#         print(dial_id, round_number, speaker, tokens)
#         print()
#         print(utterance_in_dialogue)
#         break
        
        hs = float(list(utterance_in_dialogue['normalised_h'])[0])
        hsc = float(list(utterance_in_dialogue['normalised_h_doc'])[0])
    
    new_row = list(chains.iloc[i, :].values) + [hs, hsc]
    new_df.append(tuple(new_row))
    
print(not_found)

In [ ]:
chains_dial = pd.DataFrame(new_df, columns=list(chains.keys()) + ['hs_dial', 'hsc_dial'])

In [ ]:
float(chains_dial.iloc[0, -1].values)

In [ ]:
chains_dial.head()

In [ ]:
surprisal_by_len = defaultdict(list)
for i in range(len(chains_dial)):
    surprisal_by_len[
        chains_dial.iloc[i, :]['length']
    ].append(
        chains_dial.iloc[i, :]['hs_dial']
    )

    surprisal_by_len[
        chains_dial.iloc[i, :]['length']
    ].append(
        chains_dial.iloc[i, :]['hsc_dial']
    )
    
mean_surprisal_by_len = {}
for l in surprisal_by_len:
    mean_surprisal_by_len[l] = np.mean(surprisal_by_len[l])

xu_hs_dial = []
xu_hsc_dial = []
for index, row in chains_dial.iterrows():
    try:
        xu_hs_dial.append(row['hs_dial'] / mean_surprisal_by_len[row['length']])
    except KeyError:
        xu_hs_dial.append(np.nan)
    try:
        xu_hsc_dial.append(row['hsc_dial'] / mean_surprisal_by_len[row['length']])
    except KeyError:
        xu_hsc_dial.append(np.nan)
        
chains_dial.loc[:, 'xu_hs_dial'] = xu_hs_dial
chains_dial.loc[:, 'xu_hsc_dial'] = xu_hsc_dial

df_new = chains_dial.copy()


mi_dial = []
xu_mi_dial = []
nans = 0
for index, row in chains_dial.iterrows():
    try:
        mi_dial.append(row['hs_dial'] - row['hsc_dial'])
    except KeyError:
        mi_dial.append(np.nan)
    try:
        xu_mi_dial.append(row['xu_hs_dial'] - row['xu_hsc_dial'])
    except KeyError:
        xu_mi_dial.append(np.nan)

df_new.loc[:, 'mi_dial'] = mi_dial
df_new.loc[:, 'hs_dial'] = chains_dial['hs_dial'].values
df_new.loc[:, 'hsc_dial'] = chains_dial['hsc_dial'].values
df_new.loc[:, 'xu_hs_dial'] = chains_dial['xu_hs_dial'].values
df_new.loc[:, 'xu_hsc_dial'] = chains_dial['xu_hsc_dial'].values
df_new.loc[:, 'xu_mi_dial'] = xu_mi_dial


In [ ]:
df_new.head()

In [ ]:
df_new.to_csv('/Users/mario/code/erp/chains_output/chains_mi_dial.csv')

In [39]:
chains = pd.read_csv('/Users/mario/code/erp/chains_output/chains_mi.csv').dropna()
dials = pd.read_csv('/Users/mario/Downloads/data/pb_gpt2-ft.csv').dropna()

In [61]:
chains.head()

,dialogue_id,speaker,position_in_chain,round_number,chain_id,length,tokens,xu_hs,mi,hs,hsc,xu_hsc,xu_mi
0,2389,A,1,1,1,7,"['kids', 'Ġplaying', 'Ġsoccer', 'Ġin', 'Ġgreen...",0.747428,-0.000006,3.792338,3.792345,0.747429,-0.000001
1,2389,A,2,2,1,7,"['kids', 'Ġplaying', 'Ġsoccer', 'Ġin', 'Ġgreen...",0.747428,1.806856,3.792338,1.985483,0.391317,0.356111
2,2389,B,3,4,1,10,"['4', 'Ġyoung', 'Ġboys', 'Ġin', 'Ġgreen', 'Ġje...",1.131532,0.224121,5.137746,4.913626,1.082172,0.049360
3,1969,B,1,1,2,8,"['I', 'Ġhave', 'Ġkids', 'Ġplaying', 'Ġsoccer',...",0.532963,0.000008,2.645334,2.645327,0.532961,0.000002
4,1969,A,2,3,2,3,"['s', 'Ġplaying', 'Ġsoccer']",1.618722,0.661115,11.880195,11.219079,1.528642,0.090079


In [40]:
chains.keys()

Index(['dialogue_id', 'speaker', 'position_in_chain', 'round_number',
       'chain_id', 'length', 'tokens', 'xu_hs', 'mi', 'hs', 'hsc', 'xu_hsc',
       'xu_mi'],
      dtype='object')

In [41]:
dials.keys()

Index(['dialogue_id', 'speaker', 'position_in_dialogue', 'position_in_round',
       'round_number', 'text', 'h', 'normalised_h', 'length', 'tokens_h',
       'tokens', 'xu_h', 'mi_doc', 'xu_h_doc', 'normalised_h_doc'],
      dtype='object')

In [76]:
chains.iloc[0, :].values

array([2389, 'A', 1, 1, 1, 7,
       "['kids', 'Ġplaying', 'Ġsoccer', 'Ġin', 'Ġgreen', 'Ġjerseys', '?']",
       0.7474278705465023, -6.194625581823488e-06, 3.792338460683823,
       3.792344655309405, 0.7474290914385338, -1.2208920314726868e-06],
      dtype=object)

In [101]:
hs = []
hsc = []
not_found = 0
found_multiple = defaultdict(list)

new_df = []

for i in range(len(chains)):
    dial_id = chains.iloc[i, 0]
    speaker = chains.iloc[i, 1]
    round_number = chains.iloc[i, 3]
    tokens = chains.iloc[i, 6]

    utterance_in_dialogue = dials[
        (dials['dialogue_id'] == dial_id)
        & (dials['speaker'] == speaker)
        & (dials['round_number'] == round_number)
        & (dials['tokens'] == tokens)
    ]
    
    if len(utterance_in_dialogue) == 0:
        not_found += 1
        continue
        
#     assert len(utterance_in_dialogue) == 1, f'Found {len(utterance_in_dialogue)} utterances'
    
    if len(utterance_in_dialogue) > 1:
#         print(dial_id, round_number, speaker, tokens)
#         print()
#         print(utterance_in_dialogue)

        for j, row in utterance_in_dialogue.iterrows():
            
            position_in_round = row['position_in_round']
#             print(position_in_round)
            if position_in_round in found_multiple[(dial_id, speaker, round_number, tokens)]:
#                 print('Already processed')
                continue
            else:
                found_multiple[(dial_id, speaker, round_number, tokens)].append(position_in_round)
#                 print(f'This is position {position_in_round}')
                utterance_in_dialogue = pd.DataFrame(row)
                hs = row['normalised_h']
                hsc = row['normalised_h_doc']
    else:
                
#         print(dial_id, round_number, speaker, tokens)
#         print()
#         print(utterance_in_dialogue)
#         break
        
        hs = float(list(utterance_in_dialogue['normalised_h'])[0])
        hsc = float(list(utterance_in_dialogue['normalised_h_doc'])[0])
    
    new_row = list(chains.iloc[i, :].values) + [hs, hsc]
    new_df.append(tuple(new_row))
    
print(not_found)

2642


In [102]:
chains_dial = pd.DataFrame(new_df, columns=list(chains.keys()) + ['hs_dial', 'hsc_dial'])

In [88]:
float(chains_dial.iloc[0, -1].values)

3.2162432926041737

In [103]:
chains_dial.head()

,dialogue_id,speaker,position_in_chain,round_number,chain_id,length,tokens,xu_hs,mi,hs,hsc,xu_hsc,xu_mi,hs_dial,hsc_dial
0,2389,A,1,1,1,7,"['kids', 'Ġplaying', 'Ġsoccer', 'Ġin', 'Ġgreen...",0.747428,-0.000006,3.792338,3.792345,0.747429,-0.000001,3.792338,3.216243
1,2389,A,2,2,1,7,"['kids', 'Ġplaying', 'Ġsoccer', 'Ġin', 'Ġgreen...",0.747428,1.806856,3.792338,1.985483,0.391317,0.356111,3.792338,2.197700
2,2389,B,3,4,1,10,"['4', 'Ġyoung', 'Ġboys', 'Ġin', 'Ġgreen', 'Ġje...",1.131532,0.224121,5.137746,4.913626,1.082172,0.049360,5.137746,4.661917
3,1969,B,1,1,2,8,"['I', 'Ġhave', 'Ġkids', 'Ġplaying', 'Ġsoccer',...",0.532963,0.000008,2.645334,2.645327,0.532961,0.000002,2.645336,2.645330
4,1969,A,2,3,2,3,"['s', 'Ġplaying', 'Ġsoccer']",1.618722,0.661115,11.880195,11.219079,1.528642,0.090079,11.880195,12.539207


In [109]:
surprisal_by_len = defaultdict(list)
for i in range(len(chains_dial)):
    surprisal_by_len[
        chains_dial.iloc[i, :]['length']
    ].append(
        chains_dial.iloc[i, :]['hs_dial']
    )

    surprisal_by_len[
        chains_dial.iloc[i, :]['length']
    ].append(
        chains_dial.iloc[i, :]['hsc_dial']
    )
    
mean_surprisal_by_len = {}
for l in surprisal_by_len:
    mean_surprisal_by_len[l] = np.mean(surprisal_by_len[l])

xu_hs_dial = []
xu_hsc_dial = []
for index, row in chains_dial.iterrows():
    try:
        xu_hs_dial.append(row['hs_dial'] / mean_surprisal_by_len[row['length']])
    except KeyError:
        xu_hs_dial.append(np.nan)
    try:
        xu_hsc_dial.append(row['hsc_dial'] / mean_surprisal_by_len[row['length']])
    except KeyError:
        xu_hsc_dial.append(np.nan)
        
chains_dial.loc[:, 'xu_hs_dial'] = xu_hs_dial
chains_dial.loc[:, 'xu_hsc_dial'] = xu_hsc_dial

df_new = chains_dial.copy()


mi_dial = []
xu_mi_dial = []
nans = 0
for index, row in chains_dial.iterrows():
    try:
        mi_dial.append(row['hs_dial'] - row['hsc_dial'])
    except KeyError:
        mi_dial.append(np.nan)
    try:
        xu_mi_dial.append(row['xu_hs_dial'] - row['xu_hsc_dial'])
    except KeyError:
        xu_mi_dial.append(np.nan)

df_new.loc[:, 'mi_dial'] = mi_dial
df_new.loc[:, 'hs_dial'] = chains_dial['hs_dial'].values
df_new.loc[:, 'hsc_dial'] = chains_dial['hsc_dial'].values
df_new.loc[:, 'xu_hs_dial'] = chains_dial['xu_hs_dial'].values
df_new.loc[:, 'xu_hsc_dial'] = chains_dial['xu_hsc_dial'].values
df_new.loc[:, 'xu_mi_dial'] = xu_mi_dial


In [110]:
df_new.head()

,dialogue_id,speaker,position_in_chain,round_number,chain_id,length,tokens,xu_hs,mi,hs,hsc,xu_hsc,xu_mi,hs_dial,hsc_dial,xu_hs_dial,xu_hsc_dial,mi_dial,xu_mi_dial
0,2389,A,1,1,1,7,"['kids', 'Ġplaying', 'Ġsoccer', 'Ġin', 'Ġgreen...",0.747428,-0.000006,3.792338,3.792345,0.747429,-0.000001,3.792338,3.216243,0.762816,0.646936,0.576095,0.115879
1,2389,A,2,2,1,7,"['kids', 'Ġplaying', 'Ġsoccer', 'Ġin', 'Ġgreen...",0.747428,1.806856,3.792338,1.985483,0.391317,0.356111,3.792338,2.197700,0.762816,0.442060,1.594638,0.320756
2,2389,B,3,4,1,10,"['4', 'Ġyoung', 'Ġboys', 'Ġin', 'Ġgreen', 'Ġje...",1.131532,0.224121,5.137746,4.913626,1.082172,0.049360,5.137746,4.661917,1.153996,1.047120,0.475830,0.106877
3,1969,B,1,1,2,8,"['I', 'Ġhave', 'Ġkids', 'Ġplaying', 'Ġsoccer',...",0.532963,0.000008,2.645334,2.645327,0.532961,0.000002,2.645336,2.645330,0.541597,0.541596,0.000006,0.000001
4,1969,A,2,3,2,3,"['s', 'Ġplaying', 'Ġsoccer']",1.618722,0.661115,11.880195,11.219079,1.528642,0.090079,11.880195,12.539207,1.704224,1.798760,-0.659012,-0.094536


In [111]:
df_new.to_csv('/Users/mario/code/erp/chains_output/chains_mi_dial.csv')